In [1]:
import os
os.chdir('../')

In [2]:
!find . -name .ipynb* -exec rm -rf {} \;

find: ‘./nb/.ipynb_checkpoints’: No such file or directory


In [3]:
config = """
name: "my_experiment"

# This configuration serves the purpose of documenting and explaining the settings, *NOT* as an example for good hyperparamter settings.

data: # specify your data here
    src: {lang_src}                       # src language: expected suffix of train files, e.g. "train.de"
    trg: {lang_tgt}                       # trg language
    train: {train_path}     # training data
    dev: {dev_path}         # development data for validation
    test: {test_path}       # test data for testing final model; optional
    level: {level}                  # segmentation level: either "word", "bpe" or "char"
    lowercase: True                 # lowercase the data, also for validation
    max_sent_length: 130             # filter out longer sentences from training (src+trg)
    src_voc_min_freq: 1             # src minimum frequency for a token to become part of the vocabulary
    trg_voc_min_freq: 1             # trg minimum frequency for a token to become part of the vocabulary
    #src_vocab: "my_model/src_vocab.txt"  # if specified, load a vocabulary from this file
    #trg_vocab: "my_model/trg_vocab.txt"  # one token per line, line number is index

testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 5                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search

training:                           # specify training details here
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    learning_rate: 0.0005           # initial learning rate, default: 3.0e-4 / 0.005
    learning_rate_min: 0.0001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    #learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    #learning_rate_warmup: 4000     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.                # l2 regularization, default: 0
    batch_size: 48                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token")
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 10            # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    scheduling: "plateau"           # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 500                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 15                      # train for this many epochs
    validation_freq: {val_freq}            # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    early_stopping_metric: "eval_metric"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: {model_dir} # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: True                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    max_output_length: 60           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: []    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)

model:                              # specify your model architecture here
    initializer: "xavier"           # initializer for all trainable weights (xavier, zeros, normal, uniform)
    init_weight: 0.01               # weight to initialize; for uniform, will use [-weight, weight]
    init_gain: 1.0                  # gain for Xavier initializer (default: 1.0)
    bias_initializer: "zeros"       # initializer for bias terms (xavier, zeros, normal, uniform)
    embed_initializer: "normal"     # initializer for embeddings (xavier, zeros, normal, uniform)
    embed_init_weight: 0.1          # weight to initialize; for uniform, will use [-weight, weight]
    embed_init_gain: 1.0            # gain for Xavier initializer for embeddings (default: 1.0)
    init_rnn_orthogonal: False      # use orthogonal initialization for recurrent weights (default: False)
    lstm_forget_gate: 1.            # initialize LSTM forget gate with this value (default: 1.)
    tied_embeddings: False           # tie src and trg embeddings, only applicable if vocabularies are the same, default: False
    tied_softmax: False             # tie trg embeddings and softmax (for Transformer; can be used together with tied_embeddings), default: False
    encoder:
        type: "recurrent"           # encoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        rnn_type: "gru"             # type of recurrent unit to use, either "gru" or "lstm", default: "lstm"
        embeddings:
            embedding_dim: {emb_size}      # size of embeddings
            scale: False            # scale the embeddings by sqrt of their size, default: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: {hidden_size}            # size of RNN
        bidirectional: True         # use a bi-directional encoder, default: True
        dropout: 0.3                # apply dropout to the inputs to the RNN, default: 0.0
        num_layers: 2               # stack this many layers of equal size, default: 1
        freeze: False               # if True, encoder parameters are not updated during training (does not include embedding parameters)
    decoder:
        type: "recurrent"           # decoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        rnn_type: "gru"
        embeddings:
            embedding_dim: {emb_size}
            scale: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: {hidden_size}
        dropout: 0.3
        hidden_dropout: 0.2         # apply dropout to the attention vector, default: 0.0
        num_layers: 2
        input_feeding: True         # combine hidden state and attention vector before feeding to rnn, default: True
        init_hidden: "last"         # initialized the decoder hidden state: use linear projection of last encoder state ("bridge") or simply the last state ("last") or zeros ("zero"), default: "bridge"
        attention: "bahdanau"       # attention mechanism, choices: "bahdanau" (MLP attention), "luong" (bilinear attention), default: "bahdanau"
        freeze: False               # if True, decoder parameters are not updated during training (does not include embedding parameters, but attention)
"""

In [7]:
base_path = 'data/translate/preprocessed'
datas = os.listdir(base_path)
emb_size = 300
hidden_size = 512
for data in ['Religioso']:
    data_path = os.path.join(base_path, data)
    for lang_in, lang_out in [['shp', 'es']]:
        segmentations = os.listdir(data_path)
        for segment in segmentations:
            if 'char' in segment:
                segment_path = os.path.join(data_path, segment)
                print(os.path.join('results/translate', data, segment))
                if 'bpe_drop' in segment:
                    level = 'bpe'
                elif 'bpe' in segment:
                    level = 'bpe'
                elif 'char' in segment:
                    level = 'char'
                elif 'word' in segment:
                    level = 'word'
                elif 'syl' in segment:
                    level = 'syl'
                else:
                    level = None

                val_freq = 30
                f_config = config.format(lang_src=lang_in, lang_tgt=lang_out, 
                                         train_path=os.path.join(segment_path, 'train'),
                                         test_path=os.path.join(segment_path, 'test'),
                                         dev_path=os.path.join(segment_path, 'valid'),
                                         level=level,
                                         emb_size=emb_size,
                                         hidden_size=hidden_size,
                                         val_freq=val_freq,
                                         model_dir=os.path.join('results/rnn/translate',\
                                                                f'{lang_in}-{lang_out}_{data}_{emb_size}_{hidden_size}', segment))

                with open("joeynmt/configs/transformer_test_translate.yaml",'w') as f:
                    f.write(f_config)

                !python3 joeynmt/joeynmt train "joeynmt/configs/transformer_test_translate.yaml"

results/translate/Religioso/char
2020-03-04 16:46:07,232 Hello! This is Joey-NMT.
2020-03-04 16:46:07.818876: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-04 16:46:07.818932: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-04 16:46:07.818945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-03-04 16:46:08,376 Total params: 12451936
2020-03-04 16:46:08,377 Trainable parameters: ['decoder.att_vector_layer.bias', 'decoder.att_vector_layer.weight', 'decoder.attention.energy_layer.weigh

In [ ]:
print(8)